In [193]:
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Latex


In [194]:

# Definición de variables simbólicas
x1, x2, mu = sp.symbols('x1 x2 mu')

# Función objetivo
f = x1**2 + 0.5 * x2**2

# Restricciones
g = [x1**2 + x2**2 - 1]

# Penalización basada en las restricciones
P = sum(h_j**2 for h_j in g)

# Función de penalidad
q = f + mu * P

# Gradientes y Hessiano de la función de penalidad q
variables = [x1, x2]
grad_q = [sp.diff(q, var) for var in variables]
hess_q = sp.Matrix([[sp.diff(q, var1, var2) for var1 in variables] for var2 in variables])

# Display de los gradientes y el hessiano para verificar
display(Latex("Gradientes:"))
for i, grad in enumerate(grad_q):
    display(Latex(f'$\\frac{{\\partial q}}{{\\partial x_{i+1}}} = {sp.latex(grad)}$'))

display(Latex("Hessiano:"))
for i in range(len(variables)):
    for j in range(len(variables)):
        display(Latex(f'$\\frac{{\\partial^2 q}}{{\\partial x_{i+1} \\partial x_{j+1}}} = {sp.latex(hess_q[i, j])}$'))

display(Latex(f'$H={sp.latex(hess_q)}$'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [195]:
def armijo(x0, opti, grad_q, mu_val):
    b, s, o, k1, m = 0.5, 1, 0.1, 1, 0

    while True:
        # Hallamos el valor de lambda
        lmb = s * (b ** m)
        # Hallamos f(x0)
        f = float(opti.subs({x1: x0[0], x2: x0[1], mu: mu_val}).evalf())
        # Hallamos el valor de la gradiente
        grad = sp.Matrix([grad_q[i].subs({x1: x0[0], x2: x0[1], mu: mu_val}) for i in range(len(variables))]).evalf()
        # Hallamos el valor de f(x0 - lmb * grad)
        x_new = [x0[0] - lmb * grad[0], x0[1] - lmb * grad[1]]
        f_k = float(opti.subs({x1: x_new[0], x2: x_new[1], mu: mu_val}).evalf())
        mod = float((grad[0] ** 2 + grad[1] ** 2).evalf())

        # Condición de parada
        if f_k <= float(f - o * lmb * mod):
            break
        else:
            m += 1
            k1 += 1

    return k1, lmb

In [196]:
def busqueda_exacta(x0, d_k, opti, mu_val):
    alpha = sp.symbols('alpha')
    x_new = [x0[0] + alpha * d_k[0], x0[1] + alpha * d_k[1]]
    func_new = opti.subs({x1: x_new[0], x2: x_new[1], mu: mu_val})
    dfunc_new = sp.diff(func_new, alpha)

    # Encontrar la solución de alpha que minimiza la función
    alpha_opt = sp.solve(dfunc_new, alpha)
    alpha_opt = [sol.evalf() for sol in alpha_opt if sol.is_real and sol > 0]

    if alpha_opt:
        return min(alpha_opt)
    else:
        return 1  # Valor de fallback en caso de que no se encuentre un valor óptimo


In [197]:
def gradiente_armijo(mu_val, x0, tol, max_iter):
    x = sp.Matrix(x0)
    data = []

    for k in range(max_iter):
        # Obtener la gradiente de la función de penalidad
        grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()

        # Obtener la norma del gradiente
        Normadf = float(grad.norm().evalf())

        # Condición para finalizar el programa
        if Normadf < tol:
            break

        # Obtener el número de iteraciones internas y el valor de lambda usando Armijo
        iter, lmb = armijo([x[0], x[1]], q, grad_q, mu_val)

        # Actualizar el valor de x
        x_new = (x - lmb * grad).evalf()

        # Hallar el valor de la función en x
        fun = float(q.subs({x1: x_new[0], x2: x_new[1], mu: mu_val}).evalf())

        # Error
        mod = float(x_new.norm().evalf())

        # Introducir el número de iteraciones, iteraciones internas, lambda, x0 y fun(x0)
        data.append([k, iter, x_new, fun, mod])

        # Mostrar el paso actual
        #display(Latex(f'Iteración {k}: $x = ({sp.latex(x_new[0])}, {sp.latex(x_new[1])})$, $\\nabla q = {sp.latex(grad)}$, $||\\nabla q|| = {sp.latex(Normadf)}$'))

        x = x_new

    return x, data


In [198]:
def gradiente_busqueda_exacta(mu_val, x0, tol, max_iter):
    x = sp.Matrix(x0)
    data = []

    for k in range(max_iter):
        # Obtener la gradiente de la función de penalidad
        grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()

        # Obtener la norma del gradiente
        Normadf = float(grad.norm().evalf())

        # Condición para finalizar el programa
        if Normadf < tol:
            print(f"Parando por criterio: ||grad|| < tol en la iteración {k}")
            break

        # Calcular el valor de alpha usando Búsqueda Exacta
        lmb = busqueda_exacta([x[0], x[1]], -grad, q, mu_val)
        lmb = float(lmb)  # Asegurar que alpha sea un número flotante
        print(f"Iteración {k}, alpha: {lmb}")

        # Actualizar el valor de x
        x_new = (x - lmb * grad).evalf()

        # Hallar el valor de la función en x
        fun = float(q.subs({x1: x_new[0], x2: x_new[1], mu: mu_val}).evalf())

        # Error
        mod = float(x_new.norm().evalf())

        # Introducir el número de iteraciones, iteraciones internas, lambda, x0 y fun(x0)
        data.append([k, 1, x_new, fun, mod])

        # Mostrar el paso actual
        print(f"Iteración {k}, x: {x_new}, grad: {grad}, Normadf: {Normadf}, f(x): {fun}, ||x||: {mod}")

        x = x_new

    return x, data


In [199]:
def optimize_newton_puro_armijo(mu_val, x0, tol, max_iter):
    x = sp.Matrix(x0)
    data = []

    for k in range(max_iter):
        # Calcular el gradiente y el Hessiano de la función de penalidad
        grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()
        hess = sp.Matrix([[hess_q[i, j].subs({x1: x[0], x2: x[1], mu: mu_val}) for j in range(len(variables))] for i in range(len(variables))]).evalf()

        # Calcular la dirección de Newton
        d_k = -hess.inv() * grad

        # Obtener el valor de lambda usando Armijo
        lmb = armijo_newton([x[0], x[1]], d_k, q, grad_q, mu_val)
        print(f"Iteration {k}, alpha: {lmb}")

        # Actualizar el valor de x
        x_new = (x + lmb * d_k).evalf()

        # Hallar el valor de la función en x
        fun = float(q.subs({x1: x_new[0], x2: x_new[1], mu: mu_val}).evalf())

        # Error
        mod = float(x_new.norm().evalf())

        # Introducir el número de iteraciones, iteraciones internas, lambda, x0 y fun(x0)
        data.append([k, 1, x_new, fun, mod])

        # Mostrar el paso actual
        print(f"Iteration {k}, x: {x_new}, grad: {grad}, Normadf: {grad.norm()}, f(x): {fun}, ||x||: {mod}")

        # Condición de parada
        if grad.norm() < tol:
            print(f"Parando por criterio: norm < epsilon at iteration {k}")
            break

        x = x_new

    return x, data

In [200]:
def optimize_newton_puro_exacta(mu_val, x0, tol, max_iter):
    x = sp.Matrix(x0)
    data = []

    for k in range(max_iter):
        # Calcular gradiente y hessiano
        grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()
        hess = sp.Matrix([[hess_q[i, j].subs({x1: x[0], x2: x[1], mu: mu_val}) for j in range(len(variables))] for i in range(len(variables))]).evalf()

        # Norma del gradiente
        Normadf = float(grad.norm())

        if Normadf < tol:
            break

        # Direccion de descenso
        d_k = -hess.inv() * grad

        # Calcular el valor de lambda usando Búsqueda Exacta
        lmb = busqueda_exacta([x[0], x[1]], d_k, q, mu_val)

        # Actualizar x
        x_new = (x + lmb * d_k).evalf()

        # Calcular f(x) y el error
        fun = float(q.subs({x1: x_new[0], x2: x_new[1], mu: mu_val}).evalf())
        mod = float(x_new.norm().evalf())

        data.append([k, 1, x_new.tolist(), fun, mod])

        # Mostrar el paso actual
        #display(Latex(f'Iteración {k}: $x = ({x_new[0]}, {x_new[1]})$, $\\nabla q = {grad}$, $||\\nabla q|| = {Normadf}$'))

        x = x_new

    return x, data


In [201]:
def conjugate_gradient_armijo(mu_val, x0, tol=1e-6, max_iter=1000, alpha=1, beta=0.5, sigma=1e-4):
    x = sp.Matrix(x0)
    grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()
    d = -grad
    iter_count = 0
    total_internal_iter_count = 0
    data = []

    for _ in range(max_iter):
        if grad.norm() < tol:
            break

        t = alpha
        internal_iter_count = 0
        while float(q.subs({x1: x[0] + t * d[0], x2: x[1] + t * d[1], mu: mu_val}).evalf()) > float(q.subs({x1: x[0], x2: x[1], mu: mu_val}).evalf()) + sigma * t * float((grad.T * d)[0]):
            t *= beta
            internal_iter_count += 1

        x_new = (x + t * d).evalf()
        grad_new = sp.Matrix([grad_q[i].subs({x1: x_new[0], x2: x_new[1], mu: mu_val}) for i in range(len(variables))]).evalf()
        beta_cg = (grad_new.T * grad_new)[0] / (grad.T * grad)[0]
        d = -grad_new + beta_cg * d
        x = x_new
        grad = grad_new
        iter_count += 1
        total_internal_iter_count += internal_iter_count
        data.append([iter_count, internal_iter_count, x.tolist(), float(q.subs({x1: x[0], x2: x[1], mu: mu_val}).evalf()), float(x.norm().evalf())])

    return x, iter_count, total_internal_iter_count, data


In [202]:
def conjugate_gradient_exact(mu_val, x0, tol=1e-6, max_iter=1000):
    x = sp.Matrix(x0)
    grad = sp.Matrix([grad_q[i].subs({x1: x[0], x2: x[1], mu: mu_val}) for i in range(len(variables))]).evalf()
    d = -grad
    iter_count = 0
    data = []

    for _ in range(max_iter):
        if grad.norm() < tol:
            print(f"Converged: grad.norm() < tol at iteration {iter_count}")
            break

        # Calcular el valor de alpha usando Búsqueda Exacta
        alpha = busqueda_exacta(x, d, q, mu_val)
        alpha = float(alpha)  # Asegurar que alpha sea un número flotante
        print(f"Iteration {iter_count}, alpha: {alpha}")

        x_new = (x + alpha * d).evalf()
        grad_new = sp.Matrix([grad_q[i].subs({x1: x_new[0], x2: x_new[1], mu: mu_val}) for i in range(len(variables))]).evalf()
        
        beta_num = (grad_new.T * grad_new)[0]
        beta_den = (grad.T * grad)[0]
        
        if beta_den == 0:
            beta = 0
        else:
            beta = beta_num / beta_den

        # Limitar el valor de beta para evitar pasos excesivamente grandes
        if beta > 1:
            beta = 1

        d = -grad_new + beta * d
        x = x_new
        grad = grad_new
        iter_count += 1
        data.append([iter_count, 1, x.tolist(), float(q.subs({x1: x[0], x2: x[1], mu: mu_val}).evalf()), float(x.norm().evalf())])
        
        print(f"Iteration {iter_count}, x: {x}, grad: {grad}, beta: {beta}")

    return x, iter_count, data

In [203]:
# Parámetros iniciales
x0 = [3, 0.5]  # Punto inicial factible resulta (0,-1)
x0 = [0.5, 1]  # Punto inicial factible resulta (0, 1)
#x0 = [1, 0]  # Punto inicial factible resulta (1, 0)
epsilon = 1e-3
max_iter = 100
tol = 1e-3

# Método de penalidad exterior
mu_values = []
solutions = []
all_data = []

for k in range(1, max_iter + 1):
    mu_val = k

    x_new, data =  gradiente_armijo(mu_val, x0, tol, max_iter) #Gozu

    #x_new, data =  gradiente_busqueda_exacta(mu_val, x0, tol, max_iter)

    #x_new, data = optimize_newton_puro_armijo(mu_val, x0, tol, max_iter)#Gozu

    #x_new, data = optimize_newton_puro_exacta(mu_val, x0, tol, max_iter)#Gozu
    
    #x_new, iter_count, total_internal_iter_count, data = conjugate_gradient_armijo(mu_val, x0, tol, max_iter)#gozu

    #x_new, iter_count, data = conjugate_gradient_exact(mu_val, x0, tol, max_iter)
    
    # Verificar criterio de parada
    if (sp.Matrix(x_new) - sp.Matrix(x0)).norm().evalf() < epsilon:
        print(f"Parando por criterio: norm < epsilon at iteration {k}")
        break
    
    x0 = x_new
    mu_values.append(mu_val)
    solutions.append(x0)
    all_data.extend(data)

# Mostrar los resultados
df = pd.DataFrame(all_data, columns=['Iteración', 'Iteraciones Internas', 'x', 'f(x)', '|| x - x||'])
df = df.sort_values(by='Iteración').reset_index(drop=True)
display(df)

for mu_val, sol in zip(mu_values, solutions):
    display(Latex(f'$\\mu = {mu_val}, \\; x^* = ({sol[0]}, {sol[1]})$'))


Parando por criterio: norm < epsilon at iteration 12


,Iteración,Iteraciones Internas,x,f(x),|| x - x||
0,0,4,"[0.312500000000000, 0.750000000000000]",0.494400,0.812500
1,0,4,"[0.000295836610739517, 0.974309936369953]",0.479785,0.974310
2,0,6,"[2.98915252439346e-5, 0.967402247452707]",0.484386,0.967402
3,0,6,"[1.95053333208471e-5, 0.984035008534099]",0.491186,0.984035
4,0,6,"[2.42763656001827e-5, 0.980774332543055]",0.489660,0.980774
...,...,...,...,...,...
78,14,4,"[4.67843524347744e-5, 0.935349597843497]",0.468750,0.935350
79,15,3,"[0.00105252439873444, 0.866192228186795]",0.437501,0.866193
80,16,2,"[0.000525651558436453, 0.865689691210854]",0.437500,0.865690
81,17,3,"[0.000394544115602754, 0.866192728099922]",0.437500,0.866193


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [204]:
# Definición de variables simbólicas
x1, x2, mu = sp.symbols('x1 x2 mu')

# Función objetivo para el segundo ejercicio
f = (x1 - 5)**2 + (x2 - 5)**2

# Restricciones
g = [x1**2 - x2 - 6, x1 + 3*x2 - 12, -x1, -x2]  # Usamos ecuaciones de la forma g(x) <= 0

# Penalización basada en las restricciones
P = sum(sp.Piecewise((0, h_j <= 0), (h_j**2, h_j > 0)) for h_j in g)

# Función de penalidad
q = f + mu * P

# Gradientes y Hessiano de la función de penalidad q
variables = [x1, x2]
grad_q = [sp.diff(q, var) for var in variables]
hess_q = sp.Matrix([[sp.diff(q, var1, var2) for var1 in variables] for var2 in variables])

# Display de los gradientes y el hessiano para verificar
display(Latex("Gradientes:"))
for i, grad in enumerate(grad_q):
    display(Latex(f'$\\frac{{\\partial q}}{{\\partial x_{i+1}}} = {sp.latex(grad)}$'))

display(Latex("Hessiano:"))
for i in range(len(variables)):
    for j in range(len(variables)):
        display(Latex(f'$\\frac{{\\partial^2 q}}{{\\partial x_{i+1} \\partial x_{j+1}}} = {sp.latex(hess_q[i, j])}$'))

display(Latex(f'$H = {sp.latex(hess_q)}$'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [205]:
# Parámetros iniciales
x0 = [0.5, 1.0]  # Punto inicial factible
epsilon = 1e-3
max_iter = 100
tol = 1e-3

# Método de penalidad exterior
mu_values = []
solutions = []
all_data = []

for k in range(1, max_iter + 1):
    mu_val = k

    #x_new, data =  gradiente_armijo(mu_val, x0, tol, max_iter) #Gozu

    #x_new, data =  gradiente_busqueda_exacta(mu_val, x0, tol, max_iter)

    #x_new, data = optimize_newton_puro_armijo(mu_val, x0, tol, max_iter) #gozu

    #x_new, data = optimize_newton_puro_exacta(mu_val, x0, tol, max_iter)
    
    x_new, iter_count, total_internal_iter_count, data = conjugate_gradient_armijo(mu_val, x0, tol, max_iter) #Gozu

    #x_new, iter_count, data = conjugate_gradient_exact(mu_val, x0, tol, max_iter)
    
    # Verificar criterio de parada
    if (sp.Matrix(x_new) - sp.Matrix(x0)).norm() < epsilon:
        print(f"Parando por criterio: norm < epsilon at iteration {k}")
        break
    
    x0 = x_new
    mu_values.append(mu_val)
    solutions.append(x0)
    all_data.extend(data)

# Mostrar los resultados
df2 = pd.DataFrame(all_data, columns=['Iteración', 'Iteraciones Internas', 'x', 'f(x)', '|| x - x||'])
df2 = df.sort_values(by='Iteración').reset_index(drop=True)
display(df2)

for mu_val, sol in zip(mu_values, solutions):
    display(Latex(f'$\\mu = {mu_val}, \\; x^* = ({sol[0]}, {sol[1]})$'))


Parando por criterio: norm < epsilon at iteration 16


,Iteración,Iteraciones Internas,x,f(x),|| x - x||
0,0,4,"[0.312500000000000, 0.750000000000000]",0.494400,0.812500
1,0,4,"[0.000295836610739517, 0.974309936369953]",0.479785,0.974310
2,0,6,"[2.98915252439346e-5, 0.967402247452707]",0.484386,0.967402
3,0,6,"[1.95053333208471e-5, 0.984035008534099]",0.491186,0.984035
4,0,6,"[2.42763656001827e-5, 0.980774332543055]",0.489660,0.980774
...,...,...,...,...,...
78,14,4,"[4.67843524347744e-5, 0.935349597843497]",0.468750,0.935350
79,15,3,"[0.00105252439873444, 0.866192228186795]",0.437501,0.866193
80,16,2,"[0.000525651558436453, 0.865689691210854]",0.437500,0.865690
81,17,3,"[0.000394544115602754, 0.866192728099922]",0.437500,0.866193


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>